# Evaluate Generative Classifiers on Non-Adversarial Samples

In [2]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Nicer way to import the module?
sys.path.append(str(Path.cwd().parent))
from utils.loading import load_net
from utils.display import torch_to_np, show_imgs
from models.generative_classify import optimize_latent_cvae, gen_classify_cvae, optimize_latent, gen_classify

import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torch.nn import functional as F
import torch.optim as optim

import foolbox
import json
from PIL import Image

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.current_device())

BASE_PATH = Path('/media/rene/data/adv_gen/MNIST/mnist_normal/models')

cuda:1
0


## CVAE Results

In [29]:
results= {}
results[10] = pickle.load(open(BASE_PATH / 'CVAE-1_16_32_64-16-MNIST_iter50_nt10_nsamp100_results.pkl', "rb"))
results[100] = pickle.load(open(BASE_PATH / 'CVAE-1_16_32_64-16-MNIST-10_model_best_100__results.pkl', "rb"))
results[200] = pickle.load(open(BASE_PATH / 'CVAE-1_16_32_64-16-MNIST_iter50_nt200_nsamp100_results.pkl', "rb"))
results[1000] = pickle.load(open(BASE_PATH / 'CVAE-1_16_32_64-16-MNIST_iter50_nt1000_nsamp100_results.pkl', "rb"))
results['100: newmodel'] = pickle.load(open(BASE_PATH / 'CVAE-1_16_32_64-16-MNIST_iter50_nt100_nsamp120_results.pkl', "rb"))
results['202: newmodel'] = pickle.load(open(BASE_PATH / 'CVAE-1_16_32_64-16-MNIST_iter50_nt100_nsamp202_results.pkl', "rb"))



for n_t, result in results.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Number of times: {n_t}, Accuracy {acc}')


Number of times: 10, Accuracy 0.77
Number of times: 100, Accuracy 0.88
Number of times: 200, Accuracy 0.94
Number of times: 1000, Accuracy 0.95
Number of times: 100: newmodel, Accuracy 0.8333333333333334
Number of times: 202: newmodel, Accuracy 0.31683168316831684


In [9]:
results= {}
results[10] = pickle.load(open(BASE_PATH/'CVAE-1_16_32_64-16-MNIST_iter50_nt10_nsamp200_results.pkl', "rb"))
results[25] = pickle.load(open(BASE_PATH/'CVAE-1_16_32_64-16-MNIST_iter50_nt25_nsamp200_results.pkl', "rb"))
results[50] = pickle.load(open(BASE_PATH/'CVAE-1_16_32_64-16-MNIST_iter50_nt50_nsamp200_results.pkl', "rb"))
results[100] = pickle.load(open(BASE_PATH/'CVAE-1_16_32_64-16-MNIST_iter50_nt100_nsamp200_results.pkl', "rb"))

results_d= {}
results_d[10] = pickle.load(open(BASE_PATH/'CVAE-1_16_32_64-16-MNIST_iter50_nt10_nsamp200_deter_results.pkl', "rb"))
results_d[25] = pickle.load(open(BASE_PATH/'CVAE-1_16_32_64-16-MNIST_iter50_nt25_nsamp200_deter_results.pkl', "rb"))
results_d[50] = pickle.load(open(BASE_PATH/'CVAE-1_16_32_64-16-MNIST_iter50_nt50_nsamp200_deter_results.pkl', "rb"))
results_d[100] = pickle.load(open(BASE_PATH/'CVAE-1_16_32_64-16-MNIST_iter50_nt100_nsamp200_deter_results.pkl', "rb"))

for n_t, result in results.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Number of times: {n_t}, Accuracy {acc}')
    
for n_t, result in results_d.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Deterministic: Number of times: {n_t}, Accuracy {acc}')

Number of times: 10, Accuracy 0.7
Number of times: 25, Accuracy 0.765
Number of times: 50, Accuracy 0.86
Number of times: 100, Accuracy 0.86
Deterministic: Number of times: 10, Accuracy 0.775
Deterministic: Number of times: 25, Accuracy 0.885
Deterministic: Number of times: 50, Accuracy 0.855
Deterministic: Number of times: 100, Accuracy 0.91


## Paper VAE_ABS Results

In [27]:
results= {}
results[10] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt10_nsamp100_results.pkl', "rb"))
results[100] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt100_nsamp100_results.pkl', "rb"))
results[1000] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt1000_nsamp100_results.pkl', "rb"))
results['100: newmodel'] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt100_nsamp120_results.pkl', "rb"))
results['88: old stochastic'] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt100_nsamp88_results.pkl', "rb"))


results['100: stochastic'] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt100_nsamp200_results.pkl', "rb"))
results['100: not stochastic'] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt100_nsamp202_results.pkl', "rb"))


for n_t, result in results.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Number of times: {n_t}, Accuracy {acc}')

Number of times: 10, Accuracy 0.74
Number of times: 100, Accuracy 0.86
Number of times: 1000, Accuracy 0.98
Number of times: 100: newmodel, Accuracy 0.26666666666666666
Number of times: 88: old stochastic, Accuracy 0.8977272727272727
Number of times: 100: stochastic, Accuracy 0.195
Number of times: 100: not stochastic, Accuracy 0.3811881188118812


In [7]:
results= {}
results[10] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt10_nsamp200_results.pkl', "rb"))
results[25] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt25_nsamp200_results.pkl', "rb"))
results[50] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt50_nsamp200_results.pkl', "rb"))
results[100] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt100_nsamp200_results.pkl', "rb"))

results_d= {}
results_d[10] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt10_nsamp200_deter_results.pkl', "rb"))
results_d[25] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt25_nsamp200_deter_results.pkl', "rb"))
results_d[50] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt50_nsamp200_deter_results.pkl', "rb"))
results_d[100] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt100_nsamp200_deter_results.pkl', "rb"))

for n_t, result in results.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Number of times: {n_t}, Accuracy {acc}')
    
for n_t, result in results_d.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Deterministic: Number of times: {n_t}, Accuracy {acc}')

Number of times: 10, Accuracy 0.695
Number of times: 25, Accuracy 0.835
Number of times: 50, Accuracy 0.84
Number of times: 100, Accuracy 0.89
Deterministic: Number of times: 10, Accuracy 0.84
Deterministic: Number of times: 25, Accuracy 0.855
Deterministic: Number of times: 50, Accuracy 0.895
Deterministic: Number of times: 100, Accuracy 0.94


## VAE

In [26]:
results= {}
results[10] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt10_nsamp100_results.pkl', "rb"))
results[100] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt100_nsamp100_results.pkl', "rb"))
results[200] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt200_nsamp100_results.pkl', "rb"))
results['100: newmodel'] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt100_nsamp120_results.pkl', "rb"))

results['100: stochastic'] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt100_nsamp200_results.pkl', "rb"))
results['100: not stochastic'] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt100_nsamp202_results.pkl', "rb"))



for n_t, result in results.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Number of times: {n_t}, Accuracy {acc}')

Number of times: 10, Accuracy 0.75
Number of times: 100, Accuracy 0.89
Number of times: 200, Accuracy 0.86
Number of times: 100: newmodel, Accuracy 0.19166666666666668
Number of times: 100: stochastic, Accuracy 0.17
Number of times: 100: not stochastic, Accuracy 0.31683168316831684


In [6]:
results= {}
results[10] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt10_nsamp200_results.pkl', "rb"))
results[25] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt25_nsamp200_results.pkl', "rb"))
results[50] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt50_nsamp200_results.pkl', "rb"))
results[100] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt100_nsamp200_results.pkl', "rb"))
# results[200] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt200_nsamp200_results.pkl', "rb"))


results_d= {}
results_d[10] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt10_nsamp200_deter_results.pkl', "rb"))
results_d[25] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt25_nsamp200_deter_results.pkl', "rb"))
results_d[50] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt50_nsamp200_deter_results.pkl', "rb"))
results_d[100] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt100_nsamp200_deter_results.pkl', "rb"))
# results_d[200] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt200_nsamp200_deter_results.pkl', "rb"))


for n_t, result in results.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Number of times: {n_t}, Accuracy {acc}')
    
for n_t, result in results_d.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Deterministic: Number of times: {n_t}, Accuracy {acc}')

Number of times: 10, Accuracy 0.755
Number of times: 25, Accuracy 0.765
Number of times: 50, Accuracy 0.86
Number of times: 100, Accuracy 0.875
Deterministic: Number of times: 10, Accuracy 0.805
Deterministic: Number of times: 25, Accuracy 0.85
Deterministic: Number of times: 50, Accuracy 0.9
Deterministic: Number of times: 100, Accuracy 0.91


In [19]:
results = pickle.load(open('/media/rene/data/adv_gen/MNIST/mnist_normal/models/VAE_ABS/VAE_ABS--8_iter50_nt10_nsamp2_results.pkl', "rb"))
display(results)

,0,1,2,3,4,5,6,7,8,9,path,predicted_label,true_label
0,-824.022095,-724.793457,-819.481140,-751.972412,-827.379578,-864.458496,-780.383545,-848.294617,-771.382019,-922.953613,(/media/rene/data/adv_gen/MNIST/mnist_normal/v...,1.0,6.0
1,-822.868042,-771.183838,-734.845825,-806.033447,-787.630249,-772.286621,-753.981445,-812.417603,-762.667297,-763.474915,(/media/rene/data/adv_gen/MNIST/mnist_normal/v...,2.0,6.0
